# GliaGL Evolutionary Training

Use evolutionary algorithms to discover optimal network architectures and parameters.

## What You'll Learn
- Setting up evolutionary training
- Configuring evolution parameters
- Running evolution
- Analyzing results
- Comparing with gradient-based training

In [ ]:
import glia
import numpy as np
import matplotlib.pyplot as plt

print(f"GliaGL version: {glia.__version__}")

## 1. Create Baseline Network and Data

Start with a baseline network to evolve:

In [ ]:
# Create baseline network
net = glia.Network(num_sensory=2, num_neurons=5)

# Add random connections
np.random.seed(42)
from_ids = ['S0', 'S0', 'S1', 'S1']
to_ids = ['N2', 'N3', 'N3', 'N4']
weights = np.random.randn(4)
net.set_weights(from_ids, to_ids, weights)

# Save as baseline
net.save('evolution_baseline.net')
print(f"Baseline network: {net.num_neurons} neurons, {net.num_connections} connections")

In [ ]:
# Create dataset
episodes = []
for i in range(40):
    ep = glia.EpisodeData()
    seq = glia.InputSequence()
    
    # XOR-like problem
    pattern = i % 4
    if pattern == 0:
        seq.add_timestep({'S0': 0.0, 'S1': 0.0})
        target = 'N2'
    elif pattern == 1:
        seq.add_timestep({'S0': 100.0, 'S1': 0.0})
        target = 'N3'
    elif pattern == 2:
        seq.add_timestep({'S0': 0.0, 'S1': 100.0})
        target = 'N3'
    else:
        seq.add_timestep({'S0': 100.0, 'S1': 100.0})
        target = 'N2'
    
    ep.seq = seq
    ep.target_id = target
    episodes.append(ep)

dataset = glia.Dataset(episodes)
train_data, val_data = dataset.split(0.8, seed=42)
print(f"Dataset: {len(train_data)} training, {len(val_data)} validation")

## 2. Configure Evolution

Set evolutionary algorithm parameters:

In [ ]:
# Training config for each individual
train_cfg = glia.create_config(
    lr=0.02,
    batch_size=4,
    warmup_ticks=10,
    eval_ticks=50
)

# Evolution config
evo_cfg = glia.create_evolution_config(
    population=15,        # Population size
    generations=50,       # Number of generations
    elite=3,              # Keep top 3
    parents_pool=6,       # 6 parents per generation
    train_epochs=5,       # Train each for 5 epochs
    sigma_w=0.15,         # Weight mutation strength
    sigma_thr=3.0,        # Threshold mutation
    sigma_leak=0.02,      # Leak mutation
    w_acc=1.0,            # Fitness: accuracy weight
    w_margin=0.5,         # Fitness: margin weight
    w_sparsity=0.02,      # Fitness: sparsity penalty
    lamarckian=True,      # Enable Lamarckian evolution
    seed=42
)

print(f"Evolution: {evo_cfg.population} individuals, {evo_cfg.generations} generations")

## 3. Run Evolution

This may take a few minutes:

In [ ]:
# Create evolution engine
evo = glia.Evolution(
    'evolution_baseline.net',
    train_data,
    val_data,
    train_cfg,
    evo_cfg
)

# Run evolution
print("\nRunning evolution...")
result = evo.run()

print(f"\n✓ Evolution complete!")
print(f"Best fitness: {result.best_fitness:.4f}")
print(f"Best accuracy: {result.best_acc_hist[-1]:.1%}")
print(f"Best margin: {result.best_margin_hist[-1]:.3f}")

## 4. Analyze Evolution Results

Plot fitness progression over generations:

In [ ]:
# Plot evolution history
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(result.best_fitness_hist)
axes[0].set_xlabel('Generation')
axes[0].set_ylabel('Fitness')
axes[0].set_title('Best Fitness Over Time')
axes[0].grid(True, alpha=0.3)

axes[1].plot(result.best_acc_hist)
axes[1].set_xlabel('Generation')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Best Accuracy Over Time')
axes[1].grid(True, alpha=0.3)

axes[2].plot(result.best_margin_hist)
axes[2].set_xlabel('Generation')
axes[2].set_ylabel('Margin')
axes[2].set_title('Best Margin Over Time')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Load and Test Best Network

Extract the best evolved network:

In [ ]:
# Load best genome
best_net = glia.Network()
best_net.load_from_genome(result.best_genome)
best_net.save('evolution_best.net')

print(f"Best network: {best_net.num_neurons} neurons, {best_net.num_connections} connections")

# Test on validation set
trainer = glia.Trainer(best_net, train_cfg)
correct = sum(1 for ep in val_data 
              if trainer.evaluate(ep.seq, train_cfg).winner_id == ep.target_id)
print(f"Validation accuracy: {correct/len(val_data):.1%}")

## 6. Compare with Gradient Training

Train the baseline with gradients for comparison:

In [ ]:
# Load baseline again
gradient_net = glia.Network.from_file('evolution_baseline.net')
gradient_trainer = glia.Trainer(gradient_net, train_cfg)

# Train for same number of epochs as evolution
total_epochs = evo_cfg.train_epochs * evo_cfg.generations
print(f"Training baseline with gradients for {total_epochs} epochs...")
gradient_trainer.train_epoch(train_data, epochs=total_epochs, config=train_cfg)

# Evaluate
correct = sum(1 for ep in val_data 
              if gradient_trainer.evaluate(ep.seq, train_cfg).winner_id == ep.target_id)
gradient_acc = correct / len(val_data)

print(f"\nGradient training accuracy: {gradient_acc:.1%}")
print(f"Evolution accuracy: {result.best_acc_hist[-1]:.1%}")
print(f"Improvement: {(result.best_acc_hist[-1] - gradient_acc)*100:.1f} percentage points")

## 7. Visualize Best Network

See the evolved network structure:

In [ ]:
try:
    import glia.viz as viz
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Baseline
    plt.sca(axes[0])
    viz.plot_network(glia.Network.from_file('evolution_baseline.net'), show=False)
    axes[0].set_title('Baseline Network')
    
    # Evolved
    plt.sca(axes[1])
    viz.plot_network(best_net, show=False)
    axes[1].set_title('Evolved Network')
    
    plt.tight_layout()
    plt.show()
except ImportError:
    print("Visualization requires: pip install matplotlib networkx")

## Summary

You've learned:
- ✅ Setting up evolutionary training
- ✅ Configuring evolution parameters
- ✅ Running evolution with `Evolution.run()`
- ✅ Analyzing fitness progression
- ✅ Extracting best genomes
- ✅ Comparing with gradient-based training
- ✅ Lamarckian evolution (learned → genetic)

## Key Insights

- **Evolution discovers architectures** gradient descent cannot
- **Lamarckian evolution** transfers learned weights to genome
- **Population diversity** explores solution space
- **Fitness function** balances accuracy, margin, and sparsity

## Next Steps

- **Advanced**: See `04_advanced.ipynb` for custom fitness functions
- **API Reference**: See `docs/user-guide/ADVANCED_USAGE.md`